# Shark Tank

## Imports

In [1]:
# Import required libraries
import os, json, re
import random
import asyncio
import pandas as pd, numpy as np
from tqdm import tqdm
from sharktank_utils import *
from openai import OpenAI
from pathlib import Path
from together import AsyncTogether, Together

## Read Data

In [2]:
scenario_folder = './scenario_basic_deepseek_R1'
fact_path = 'facts_shark_tank_transcript_28_HummViewer.txt'
scenario_name = "_".join(fact_path.split('_')[1:])

# Read the facts
facts_store = load_facts()
# fact_dict = facts_store[fact_path]

In [ ]:
groq_api_key = os.environ['GROQ_API_KEY']

['GROQ_API_KEY', 'TOGETHER_API_KEY']

## MultiAgent Functions
- Multiagent Framework #1
> Prompt --> Orchestrator --> N * Agents --> Aggregator --> Output

In [8]:
# Define model array
reference_models = [
    "Gryphe/MythoMax-L2-13b-Lite",
    "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free", # Free
    "mistralai/Mistral-7B-Instruct-v0.3", # Free
    "scb10x/scb10x-llama3-1-typhoon-18370",
    "deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free", # Free
    'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo'
]
aggregator_model = "mistralai/Mistral-7B-Instruct-v0.3"
evaluator_model = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

# Set the system level prompt
NUM_AGENTS = 3
taskmaster_system_prompt = f"""
You are a sharktank pitch director. You will take control {NUM_AGENTS} other LLM agents like yourself. 
You will be given a fact_dict containing the key facts of the product you will be creating the pitch for.
Breakdown the task into 3 tasks and assign each portion to each LLM agent.
Output your answer in a messages list that can be parsed by python's `eval()`. For example: 
{{
    "agent1" : [{{"role":"system", "content":"<insert role>"}}, {{"role":"user","content":"<insert prompt>"}}],
    "agent2" : [{{"role":"system", "content":"<insert role>"}}, {{"role":"user","content":"<insert prompt>"}}],
    "agent3" : [{{"role":"system", "content":"<insert role>"}}, {{"role":"user","content":"<insert prompt>"}}],
}}
Create winning pitches! No hallucinations! No explanations of the pitch!
"""
aggregator_system_prompt  = """
You have been provided with a set of responses from various open-source models to the latest user query. 
Your task is to synthesize these responses into a single, high-quality response. Output a script for prospective entrepreneurs to use.
It is crucial to critically evaluate the information provided in these responses,
recognizing that some of it may be biased or incorrect. 
Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. 
Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.
Responses from models:"""

# Define the client for use
client = Together(
    api_key="cfdb186041125ce240c532fd6f26db63bde4cb3811118f52e47508e5f0398836"
)

In [4]:
def generate_user_prompt(facts):
    return f"""
My name is Paul McCarthney
Here are the facts of my product: {facts}
"""

def select_models(num_agents=NUM_AGENTS, reference_models=reference_models):
    return random.choices(reference_models, k=num_agents)

def generate_pitch(facts, num_agents=NUM_AGENTS, reference_models=reference_models):
    """Run the main loop of the MOA process."""
    prompt = generate_user_prompt(facts)
    assignments = eval(run_llm(
        client=client,
        model=aggregator_model,
        system_prompt=taskmaster_system_prompt,
        user_prompt=prompt
    ))
    assignments_list = list(assignments.values())

    results = [run_llm(
        client=client,
        model=reference_models[i],
        system_prompt=assignments_list[i][0]['content'],
        user_prompt=assignments_list[i][1]['content']
    ) for i in range(num_agents)]

    final_pitch = client.chat.completions.create(
        model=aggregator_model,
        messages=[
            {"role": "system","content": aggregator_system_prompt},
            {"role": "user", "content": aggregate_results(results, prompt)},
        ]
    )
    return final_pitch

In [16]:
# Create a fixed set of agents:
agents = select_models()

pitches = {}
for name, fact in facts_store:
    pitch = generate_pitch(
        fact, 
        num_agents=NUM_AGENTS, 
        reference_models=reference_models
    ).choices[0].message.content

    pitches[name] = pitch

ChatCompletionResponse(id='nmEXmNA-3NKUce-9213db5a3e3e91b7', object=<ObjectType.ChatCompletion: 'chat.completion'>, created=1742123979, model='mistralai/Mistral-7B-Instruct-v0.3', choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=2101572877195609300, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' {\n  "pitch": {\n    "delivery": "The entrepreneurs, Pete and Bianca, presented their pitch with a narrative involving a character named Joe, illustrating the daily exposure to germs and the lack of solutions for cleaning clothes on the go. They used a demonstration to show the product\'s effectiveness. The pitch was delivered with enthusiasm and a clear passion for solving a real-world problem. The entrepreneurs were confident and engaged the Sharks with a story that highlighted the necessity of their product.",\n    "sentiment": "The pitch was delivered with enthusiasm and a clear passion for

In [ ]:
# Output the dictionary as a JSON object
with open('basic_multiagent_pitches.json', 'w') as f:
    json.dumps(pitches)

Network based collaboration generation
https://arxiv.org/pdf/2502.11098v1
> Prompt --> Supervisor --> Generate --> Evaluate --> Repeat --> Output

In [5]:
editor_prompt = f"""
You are a pitch editor. You will be given a pitch. Evaluate its strength. 
Give constructive feedback on how to improve the pitch a for shark tank pitch.
Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.
Here is the pitch:
"""

In [6]:
def generate_feedback_pitch(facts, num_agents=NUM_AGENTS, reference_models=reference_models, loops=3):
    """Run the main loop of the MOA process."""
    initial_pitch, feedback = None, None
    for l in tqdm(range(loops)):
        prompt = generate_user_prompt(facts)
        if initial_pitch:
            prompt += f"""
            Here was your previous attempt: {initial_pitch}. Improve upon it.
            """
        
        if feedback:
            prompt += f"""
            This is the feedback of your previous attempt: {feedback}
            """
        assignments = eval(run_llm(
            client=client,
            model=aggregator_model,
            system_prompt=taskmaster_system_prompt,
            user_prompt=prompt
        ))
        assignments_list = list(assignments.values())

        results = [
            run_llm(
                client=client,
                model=reference_models[i],
                system_prompt=assignments_list[i][0]['content'],
                user_prompt=assignments_list[i][1]['content']
            ) for i in range(num_agents)
        ]

        final_pitch = run_llm(
            client=client,
            model=aggregator_model,
            system_prompt=aggregator_system_prompt,
            user_prompt=aggregate_results(results, prompt)
        )

        if l < loops-1:
            evaluation = run_llm(
                client=client,
                model=evaluator_model,
                system_prompt=editor_prompt,
                user_prompt=final_pitch,
                temperature=0.7
            )

            initial_pitch = final_pitch
            feedback = evaluation
    return final_pitch

In [9]:
# Create a fixed set of agents:
agents = select_models()

pitches = {}
# for name, fact in facts_store:
sample = list(facts_store.keys())[0]
name, fact = sample, facts_store[sample]
pitch = generate_feedback_pitch(
    fact, 
    num_agents=NUM_AGENTS, 
    reference_models=reference_models
)
print(pitch)

pitches[name] = pitch

100%|██████████| 3/3 [01:15<00:00, 25.07s/it]

 {
  "pitch": {
    "delivery": "Welcome to GarmaGuard, a revolutionary fabric cleanser designed for busy lifestyles. Our product uses natural propellants to eliminate odor, freshen fabric, and control dirt and grime, all while killing 99% of germs and odor-causing bacteria. With a strong sales performance of $476,000 over 1.5 years, a projected sales target of $500,000 this year, a 20% profit margin, and 14,000 loyal customers, we are poised to revolutionize the fabric care industry. Our unique selling point is our on-the-go convenience, eco-friendliness, and effectiveness in eliminating odors and bacteria. To achieve these impressive results, we have implemented a comprehensive marketing and sales strategy, including digital marketing, content marketing, influencer marketing, paid advertising, event marketing, account-based sales, solution selling, consultative selling, referral and word-of-mouth marketing, data-driven sales, customer relationship management, sales enablement, pricin